In [1]:
import tensorflow as tf
tf.__version__

2024-11-14 16:03:33.557332: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-14 16:03:33.584873: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-14 16:03:33.585695: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-14 16:03:34.155253: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


'2.13.0'

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


2024-11-14 15:23:11.693567: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-14 15:23:11.858929: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-14 15:23:11.858981: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
import os
import random
import itertools
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.layers import Input, GlobalAveragePooling1D, Concatenate, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K

from transformers import AutoTokenizer, TFLongformerModel, LongformerConfig, AutoModel, BertConfig, TFAutoModel, TFBertModel, LongformerTokenizer

/root/miniconda3/envs/longnew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=25)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=15)
    plt.yticks(tick_marks, classes, fontsize=15)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black", fontsize = 14)

    plt.ylabel('True label', fontsize=20)
    plt.xlabel('Predicted label', fontsize=20)

In [5]:
import pandas as pd

df = pd.read_csv('FinalDatasetBalanced.csv')

In [6]:
df.head()

,source_content,suspicious_content,plagiarism_type
0,بين عالم يعمل على أن يجعل الحياة أكثر رفاهية، ...,إن الاختلاج هو الحالة التي يحدث فيها تفريغ فجا...,No Plagiarism
1,لماذا نحتاج لطرح صيغة جديدة من صيغ الإجابة على...,اً كانوا يؤدونه للنبي صلى الله عليه وسلم لقاتل...,No Plagiarism
2,* اختارتني الأمم المتحدة سفيرا لانحيازي للبسطا...,"المخرجة الفلسطينية ""مي المصري"" في مجمل أعمالها...",No Plagiarism
3,"الحياة محطات.\nوحياتي تزخر بمحطات كثيرة, في ال...","وة إلى ""أمة الإسلام""، وكان في دعوته يميل إلى ا...",No Plagiarism
4,لمحات من سيرة عذبة لواحد من عشاق التراث العربي...,ن الإسلام.. يقدمه - عن فهم وعقيدة - على أنه نظ...,No Plagiarism


In [7]:
df['plagiarism_type'].unique()

array(['No Plagiarism', 'Artificial Obfuscation', 'No Obfuscation',
       'Simulated Obfuscation'], dtype=object)

In [8]:
df['plagiarism_type'] = df['plagiarism_type'].factorize()[0]

In [9]:
map_label = dict(enumerate(df['plagiarism_type'].factorize()[1]))


In [10]:
map_label

{0: 0, 1: 1, 2: 2, 3: 3}

In [11]:
df['plagiarism_type'].unique()

array([0, 1, 2, 3])

In [12]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    
def convert_to_transformer_inputs(str1, str2, tokenizer, max_sequence_length, double=True):
    def return_id(str1, str2, length):
        inputs = tokenizer.encode_plus(
            str1, str2,
            add_special_tokens=True,
            max_length=length,
            truncation=True,
            return_token_type_ids=True
        )
        input_ids = inputs["input_ids"]
        input_masks = inputs["attention_mask"]
        input_segments = inputs["token_type_ids"]
        
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks, input_segments]

    if double:
        input_ids_1, input_masks_1, input_segments_1 = return_id(str1, None, max_sequence_length)
        input_ids_2, input_masks_2, input_segments_2 = return_id(str2, None, max_sequence_length)

        return [input_ids_1, input_masks_1, input_segments_1,
                input_ids_2, input_masks_2, input_segments_2]
    else:
        input_ids, input_masks, input_segments = return_id(str1, str2, max_sequence_length)

        return [input_ids, input_masks, input_segments, None, None, None]

def compute_input_arrays(df, columns, tokenizer, max_sequence_length, double=True):
    input_ids_1, input_masks_1, input_segments_1 = [], [], []
    input_ids_2, input_masks_2, input_segments_2 = [], [], []
    
    for _, instance in df[columns].iterrows():
        str1, str2 = instance[columns[0]], instance[columns[1]]
        ids_1, masks_1, segments_1, ids_2, masks_2, segments_2 = \
            convert_to_transformer_inputs(str1, str2, tokenizer, max_sequence_length, double=double)
        
        input_ids_1.append(ids_1)
        input_masks_1.append(masks_1)
        input_segments_1.append(segments_1)
        input_ids_2.append(ids_2)
        input_masks_2.append(masks_2)
        input_segments_2.append(segments_2)

    if double:
        return [np.asarray(input_ids_1, dtype=np.int32), 
                np.asarray(input_masks_1, dtype=np.int32), 
                np.asarray(input_segments_1, dtype=np.int32),
                np.asarray(input_ids_2, dtype=np.int32), 
                np.asarray(input_masks_2, dtype=np.int32), 
                np.asarray(input_segments_2, dtype=np.int32)]
    else:
        return [np.asarray(input_ids_1, dtype=np.int32), 
                np.asarray(input_masks_1, dtype=np.int32), 
                np.asarray(input_segments_1, dtype=np.int32)]

In [13]:
### TRAIN TEST SPLIT ###

X_train, X_test, y_train, y_test = train_test_split(df[['source_content','suspicious_content']], df['plagiarism_type'].values, 
                                                    random_state=33, test_size = 0.3)

# del df

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(1702, 2) (730, 2)
(1702,) (730,)


In [14]:
### IMPORT TOKENIZER ###

MAX_SEQUENCE_LENGTH = 16000
MODEL_NAME = "longformer-encdec-base-16384"
# MODEL_NAME = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# **DUAL BERT (TWO INPUT)**

In [15]:
### CREATE SEQUENCES (id, mask, segments) FOR TRAIN AND TEST ###

input_train = compute_input_arrays(X_train,['source_content','suspicious_content'], tokenizer, MAX_SEQUENCE_LENGTH)
input_test = compute_input_arrays(X_test, ['source_content','suspicious_content'], tokenizer, MAX_SEQUENCE_LENGTH)

In [ ]:
def dual_longformer():
    
    set_seed(33)
    opt = Adam(learning_rate=2e-5)
    
    id1 = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    id2 = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    mask1 = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    mask2 = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    atn1 = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    atn2 = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    config = LongformerConfig.from_pretrained(MODEL_NAME)
    config.max_position_embeddings = 16000
    config.attention_window = [256] * config.num_hidden_layers

    # tf.debugging.set_log_device_placement(True)


    # with tf.device('/GPU:0'):
    longformer_model1 = TFAutoModel.from_pretrained(MODEL_NAME, config=config, from_pt=True)
    longformer_model2 = TFAutoModel.from_pretrained(MODEL_NAME, config=config, from_pt=True)
    
    embedding1 = longformer_model1(id1, attention_mask=mask1, token_type_ids=atn1)[0]
    embedding2 = longformer_model2(id2, attention_mask=mask2, token_type_ids=atn2)[0]
    
    x1 = GlobalAveragePooling1D()(embedding1)
    x2 = GlobalAveragePooling1D()(embedding2)
    
    x = Concatenate()([x1, x2])
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    out = Dense(len(map_label), activation='softmax')(x)

    model = Model(inputs=[id1, mask1, atn1, id2, mask2, atn2], outputs=out)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)
    
    return model 

In [ ]:
class SavePretrainedCallback(tf.keras.callbacks.Callback):
    def __init__(self, save_path='longformer_checkpoints'):
        super().__init__()
        self.save_path = save_path
        
    def on_epoch_end(self, epoch, logs=None):
        epoch_path = f'{self.save_path}/epoch_{epoch+1}'
        os.makedirs(epoch_path, exist_ok=True)
        
        # Save the model
        self.model.save_pretrained(epoch_path)
        print(f'\nSaved model to {epoch_path}')


: 

In [ ]:
model = dual_longformer()
history = model.fit(input_train, y_train, epochs=4, batch_size=2, validation_data=(input_test, y_test),
    verbose=1,callbacks=[SavePretrainedCallback()])

2024-11-14 15:24:03.692447: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-14 15:24:03.692540: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-14 15:24:03.692572: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-14 15:24:03.952846: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-14 15:24:03.952919: I tensorflow/compile

Epoch 1/4


In [ ]:
# Plot training history
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
### PREDICT TEST ###

pred_test = np.argmax(model.predict(input_test), axis=1)

In [ ]:
print(classification_report([map_label[i] for i in y_test], [map_label[i] for i in pred_test]))

In [ ]:
cnf_matrix = confusion_matrix([map_label[i] for i in y_test], 
                              [map_label[i] for i in pred_test])

plt.figure(figsize=(7,7))
plot_confusion_matrix(cnf_matrix, classes=list(map_label.values()))
plt.show()

# **SIAMESE BERT (TWO INPUT)**

In [ ]:
def siamese_longformer():
    
    set_seed(33)
    
    opt = Adam(learning_rate=2e-5)
    
    id1 = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    id2 = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    mask1 = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    mask2 = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    atn1 = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    atn2 = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    config = LongformerConfig()
    config.output_hidden_states = False # Set to True to obtain hidden states
    
    tf.debugging.set_log_device_placement(True)


    with tf.device('/GPU:0'):
        longformer_model = TFLongformerModel.from_pretrained(MODEL_NAME,from_pt=True, config=config)

    embedding1 = longformer_model(id1, attention_mask=mask1, token_type_ids=atn1)[0]
    embedding2 = longformer_model(id2, attention_mask=mask2, token_type_ids=atn2)[0]
    
    x1 = GlobalAveragePooling1D()(embedding1)
    x2 = GlobalAveragePooling1D()(embedding2)
    
    x = Concatenate()([x1, x2])
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    out = Dense(len(map_label), activation='softmax')(x)

    model = Model(inputs=[id1, mask1, atn1, id2, mask2, atn2], outputs=out)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)
    
    return model

: 

In [ ]:
model = siamese_longformer()
history = model.fit(input_train, y_train, epochs=3, batch_size=1, validation_data=(input_test, y_test),
    verbose=1,callbacks=[SavePretrainedCallback()])

2024-11-14 15:20:12.567041: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-14 15:20:12.567304: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-14 15:20:12.567339: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-14 15:20:12.964738: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-14 15:20:12.964898: I tensorflow/compile

Epoch 1/3


In [ ]:
# Plot training history
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
### PREDICT TEST ###

pred_test = np.argmax(model.predict(input_test), axis=1)

In [ ]:
print(classification_report([map_label[i] for i in y_test], [map_label[i] for i in pred_test]))

In [ ]:
cnf_matrix = confusion_matrix([map_label[i] for i in y_test], 
                              [map_label[i] for i in pred_test])

plt.figure(figsize=(7,7))
plot_confusion_matrix(cnf_matrix, classes=list(map_label.values()))
plt.show()